In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import re

In [2]:
df=pd.read_excel("todos.xlsx")
df2=pd.read_excel("com1.xlsx")

In [5]:
scaler=MinMaxScaler()
df["fullname"]=df["Nombre"] + df["Apellido(s)"]
df2["fullname"]=df2["Nombre"] + df2["Apellido(s)"]
df["hash"]=df.fullname.map(lambda x: hash(x))
df2["hash"]=df2.fullname.map(lambda x: hash(x))
parcial_com1=pd.concat([df[df.hash==code] for code in df2.hash])
parcial_com1=parcial_com1.iloc[:,~parcial_com1.replace({"-":np.nan}).isnull().all().values[..., np.newaxis]].copy()
parcial_com1.drop(columns=['Nombre', 'Apellido(s)', 'Número de ID', 'Dirección de correo','Total del curso (Real)', 'Última descarga de este curso', 'fullname'], inplace=True)
parcial_com1.replace("-","0", inplace=True)
evaluaciones_com1=df2.iloc[:,~df2.replace({"-":np.nan}).isnull().all().values[..., np.newaxis]].copy()
evaluaciones_com1.drop(columns=['Nombre', 'Apellido(s)', 'Número de ID', 'Dirección de correo','Total del curso (Real)', 'Última descarga de este curso', 'fullname'], inplace=True)
evaluaciones_com1.replace("-","0", inplace=True)
datacols=[col for col in evaluaciones_com1 if "hash" not in col]
data_evs=scaler.fit_transform(evaluaciones_com1[datacols])
todas_evs_c1=pd.DataFrame(data=data_evs, index=evaluaciones_com1.hash,columns=datacols)
ev_control=todas_evs_c1.loc[:,[col for col in todas_evs_c1.columns if "Control" in col]]
datacols_p=[col for col in parcial_com1 if "hash" not in col]
data_parc=scaler.fit_transform(parcial_com1[datacols_p])
parcial_c1=pd.DataFrame(data=data_parc, index=parcial_com1.hash,columns=datacols_p)

ev_control.to_csv("ev_control.csv")
parcial_c1.to_csv("parcial_c1.csv")
